In [1]:
# Import Libraries
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from ta.others import daily_return,cumulative_return
from plotly.offline import init_notebook_mode
import cufflinks as cf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import requests
from bs4 import BeautifulSoup
import warnings
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
%matplotlib inline
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
# Check if folder input exists

# Define the folder name
folder_name = '../input'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../input' folder already exists.


In [3]:
# Check if folder tickers exists

# Define the folder name
folder_name = '../tickers'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../tickers' folder already exists.


In [4]:
# Webscrapping Wikipedia table to get the IBEX 35 Growth components
page = requests.get("https://es.wikipedia.org/wiki/BME_Growth")
soup = BeautifulSoup(page.text, 'html.parser') 
table = soup.find('table',class_="wikitable sortable")

In [5]:
# Convert html table to dataframe
wiki = pd.read_html(str(table))
wiki = pd.concat(wiki)
wiki.head(40)

,Ticker,Empresa,Sede,Sector[7]​,Contratación[7]​,ISIN
0,ADL.MC,ADL Bionatur Solutions,Jerez de la Frontera,Productos farmacéuticos y biotecnología,Continuo,ES0184980003
1,AGIL.MC,Agile Content,Madrid,Electrónica y software,Continuo,ES0105102000
2,COM.MC,Catenon,Madrid,Electrónica y software,Continuo,ES0112320009
3,EIDF.MC,EiDF,Pontevedra,Energías renovables,Continuo,ES0105517009
4,ELZ.MC,Asturiana de Laminados (elZinc),Lena,"Mineral, metales y transformación",Continuo,ES0105227005
5,END.MC,Endurance Motive,Canet de Berenguer,Fabricación y montaje de bienes de equipo,Continuo,ES0105589008
6,FACE.MC,FacePhi Biometria,Alicante,Electrónica y software,Continuo,ES0105029005
7,GIGA.MC,Gigas Hosting,Alcobendas,Electrónica y software,Continuo,ES0105093001
8,GRN.MC,Greenalia,La Coruña,Energías renovables,Continuo,ES0105293007
9,HLZ.MC,Holaluz,Barcelona,Energías renovables,Continuo,ES0105456026


In [6]:
# Unfortunately Greenalia and ADL do not have tickers in Yahoo finance
tickers_to_drop = ['ADL.MC', 'GRN.MC','TR1.MC']
wiki = wiki[~wiki['Ticker'].isin(tickers_to_drop)]
wiki

,Ticker,Empresa,Sede,Sector[7]​,Contratación[7]​,ISIN
1,AGIL.MC,Agile Content,Madrid,Electrónica y software,Continuo,ES0105102000
2,COM.MC,Catenon,Madrid,Electrónica y software,Continuo,ES0112320009
3,EIDF.MC,EiDF,Pontevedra,Energías renovables,Continuo,ES0105517009
4,ELZ.MC,Asturiana de Laminados (elZinc),Lena,"Mineral, metales y transformación",Continuo,ES0105227005
5,END.MC,Endurance Motive,Canet de Berenguer,Fabricación y montaje de bienes de equipo,Continuo,ES0105589008
6,FACE.MC,FacePhi Biometria,Alicante,Electrónica y software,Continuo,ES0105029005
7,GIGA.MC,Gigas Hosting,Alcobendas,Electrónica y software,Continuo,ES0105093001
9,HLZ.MC,Holaluz,Barcelona,Energías renovables,Continuo,ES0105456026
10,IZER.MC,Izertis,Gijón,Telecomunicaciones y otros,Continuo,ES0105449005
11,LLN.MC,Lleida.net,Lérida,Telecomunicaciones y otros,Continuo,ES0105089009


In [7]:
wiki.rename(columns={'Empresa':'Name','Sector[7]\u200b':'Sector'}, inplace=True)

In [8]:
# Get the data for the stock index
index_list = wiki['Ticker'].tolist()

In [9]:
# Save all the historical data
for stock in tqdm(index_list):
    data = yf.download(stock, progress=False,multi_level_index=False,actions=True,auto_adjust=False)
    data.to_csv(f"../input/{stock}.csv",index=True)

  0%|          | 0/12 [00:00<?, ?it/s]

In [10]:
# Get the name of the notebook
notebook_name = os.path.basename(globals()['__vsc_ipynb_file__'])
notebook_name = notebook_name.split('-')[0]

In [11]:
# Save all the tickers data
wiki[['Name','Sector','Ticker']].to_csv(f"../tickers/{notebook_name}.csv",index=True)

In [12]:
# Configuration of different parameters of the notebook
ticker = 'HLZ.MC'
year = '2025'

In [13]:
# Check DataFrame
stock_ticker = pd.read_csv(f"../input/{ticker}.csv",index_col="Date",parse_dates=True)
stock_ticker.head(10)

,Adj Close,Close,Dividends,High,Low,Open,Stock Splits,Volume
Date,,,,,,,,
2020-07-30,7.00,7.00,0.0,7.14,7.00,7.00,0.0,7455
2020-07-31,7.10,7.10,0.0,7.15,7.00,7.15,0.0,7801
2020-08-03,7.00,7.00,0.0,7.14,6.95,7.01,0.0,5465
2020-08-04,7.00,7.00,0.0,7.10,6.85,7.00,0.0,1714
2020-08-05,7.09,7.09,0.0,7.09,7.05,7.09,0.0,1066
2020-08-06,7.10,7.10,0.0,7.15,6.80,7.09,0.0,9197
2020-08-07,6.92,6.92,0.0,7.15,6.92,7.15,0.0,9668
2020-08-10,6.92,6.92,0.0,6.96,6.96,6.96,0.0,145
2020-08-11,7.06,7.06,0.0,7.10,6.92,6.92,0.0,1673


In [14]:
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,vertical_spacing=0.01,specs=[[{'rowspan':3,'colspan':1}],[None],[None],[{'rowspan':1,'colspan':1}]])

# Graph (1,1)
fig.add_trace(go.Scatter(x=stock_ticker.index,y=stock_ticker['Close'],mode="lines",name=f"{ticker}"),row=1, col=1)
# Update xaxis properties
fig.update_yaxes(title_text="Price", row=1, col=1)

# Graph (4,1)
fig.add_trace(go.Scatter(x=stock_ticker.index,y=stock_ticker['Volume'],mode="lines",name='Volume'),row=4, col=1)
fig.update_yaxes(title_text="Volume", row=4, col=1)

fig.update_layout(height=800, width=1300,showlegend=False,title=f"{ticker}")

fig.show()

In [15]:
# Add Technical Analysis Indicators

# Modified Moving Average 20
stock_ticker['MMA20'] = stock_ticker['Adj Close'].loc[year].rolling(20).mean() #Adj Close 20 MA

# Initialize Bollinger Bands Indicator
indicator_bb = BollingerBands(close=stock_ticker["Adj Close"].loc[year], window=20, window_dev=2)

# Bollinger Bands
stock_ticker['BB_Upper'] = indicator_bb.bollinger_hband()
stock_ticker['BB_Lower'] = indicator_bb.bollinger_lband()

# Initialize RSI Indicator
indicator_rsi = RSIIndicator(close=stock_ticker["Adj Close"].loc[year], window=14)

# RSI
stock_ticker['RSI'] = indicator_rsi.rsi()

# Daily Return
stock_ticker['Daily_Return'] = daily_return(stock_ticker["Adj Close"].loc[year])

In [16]:
# Plot the adjusted close price
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,vertical_spacing=0.01,specs=[[{'rowspan':2,'colspan':1}],[None],[{'rowspan':1,'colspan':1}],[{'rowspan':1,'colspan':1}]])

# Graph (1,1)
fig.add_trace(go.Scatter(x=stock_ticker['Adj Close'].loc[year].index,y=stock_ticker['Adj Close'].loc[year],mode="lines",name=f'{ticker}'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['BB_Lower'].loc[year].index,y=stock_ticker['BB_Lower'].loc[year],mode="lines",name='BB_Lower'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['BB_Upper'].loc[year].index,y=stock_ticker['BB_Upper'].loc[year],mode="lines",name='BB_Upper'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['MMA20'].loc[year].index,y=stock_ticker['MMA20'].loc[year],mode="lines",name='MMA20'),row=1,col=1)
# Update xaxis properties
fig.update_yaxes(title_text="Price", row=1, col=1)

# Graph (3,1)
fig.add_trace(go.Scatter(x=stock_ticker['Volume'].loc[year].index,y=stock_ticker['Volume'].loc[year],mode="lines",name='Volume'),row=3, col=1)
fig.add_trace(go.Scatter(x=stock_ticker['Volume'].loc[year].index,y=stock_ticker['Volume'].loc[year].rolling(20).mean(),mode="lines",name='MMA20'),row=3,col=1)
fig.update_yaxes(title_text="Volume", row=3, col=1)

# Graph (4,1)
fig.add_trace(go.Scatter(x=stock_ticker['RSI'].loc[year].index,y=stock_ticker['RSI'].loc[year],mode="lines",name='RSI'),row=4, col=1)
fig.add_hline(y=30, line_width=1, line_dash="dash", line_color="green",row=4,col=1)
fig.add_hline(y=70, line_width=1, line_dash="dash", line_color="red",row=4,col=1)
fig.update_yaxes(title_text="RSI", row=4, col=1)

fig.update_layout(height=800, width=1300,showlegend=False,title=f"{ticker} {year}")

fig.show()

In [17]:
# Plotly
fig = make_subplots()

# Graph (1,1)
# Loop all stock files and get cummulative return for year
for stock in tqdm(index_list):
    f = os.path.join("../input", stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    fig.add_trace(go.Scatter(x=df['Cummulative_Return'].loc[year].index,y=df['Cummulative_Return'].loc[year],mode="lines",name=stock.split('.')[0]),row=1,col=1)

# Update xaxis properties
fig.update_yaxes(title_text="Return", row=1, col=1)

fig.update_layout(height=800, width=1300,showlegend=True,title=f"Cummulative Returns {ticker} for {year}")

fig.show()

  0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
# Create also table of cummulative returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input", stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Cummulative_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Cummulative_Return'])
cum = wiki[['Ticker','Name']].merge(cum,on='Ticker')
cum.sort_values(by=['Cummulative_Return'],ignore_index=True, ascending=False)

  0%|          | 0/12 [00:00<?, ?it/s]

,Ticker,Name,Date,Cummulative_Return
0,COM.MC,Catenon,2025-05-23,50.375935
1,END.MC,Endurance Motive,2025-05-23,49.999995
2,FACE.MC,FacePhi Biometria,2025-05-23,35.816612
3,EIDF.MC,EiDF,2025-05-23,25.118486
4,ELZ.MC,Asturiana de Laminados (elZinc),2025-05-23,19.517539
5,LLN.MC,Lleida.net,2025-05-23,11.574074
6,SNG.MC,SNGULAR,2025-05-23,0.062890
7,HLZ.MC,Holaluz,2025-05-23,-1.167314
8,PAR.MC,Parlem Telecom,2025-05-23,-4.117650
9,IZER.MC,Izertis,2025-05-23,-7.024796


In [19]:
# Create also table of daily returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input",stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Daily_Return'] = daily_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Daily_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Daily_Return'])
cum = wiki[['Ticker','Name']].merge(cum,on='Ticker')
cum.sort_values(by=['Daily_Return'],ignore_index=True, ascending=False)

  0%|          | 0/12 [00:00<?, ?it/s]

,Ticker,Name,Date,Daily_Return
0,HLZ.MC,Holaluz,2025-05-23,7.627122
1,END.MC,Endurance Motive,2025-05-23,4.807696
2,AGIL.MC,Agile Content,2025-05-23,0.000000
3,GIGA.MC,Gigas Hosting,2025-05-23,0.000000
4,PAR.MC,Parlem Telecom,2025-05-23,0.000000
5,SNG.MC,SNGULAR,2025-05-23,0.000000
6,FACE.MC,FacePhi Biometria,2025-05-23,-0.420178
7,COM.MC,Catenon,2025-05-23,-0.990098
8,EIDF.MC,EiDF,2025-05-23,-0.999999
9,IZER.MC,Izertis,2025-05-23,-1.098905
